In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('tickets.csv')

##### Head

In [3]:
print(df.head())

   ticket_id       _fivetran_synced        assignee_id           brand_id  \
0        220  abril 20, 2022, 14:53  4,531,543,131,803  4,705,224,012,571   
1        222  abril 20, 2022, 14:53  4,623,907,812,507  4,705,224,012,571   
2  1,301,191  abril 20, 2022, 15:19  4,623,907,810,715  4,705,210,809,883   
3  1,408,221   mayo 19, 2022, 14:12  4,623,907,810,715  4,705,210,809,883   
4  1,427,909  julio 31, 2022, 08:14  4,623,907,810,715  4,705,210,809,883   

              created_at             updated_at  \
0  marzo 25, 2022, 19:47   abril 8, 2022, 00:08   
1  marzo 25, 2022, 19:50   abril 8, 2022, 00:20   
2  abril 12, 2022, 15:42  abril 16, 2022, 00:03   
3   mayo 16, 2022, 19:18   mayo 19, 2022, 14:04   
4  julio 27, 2022, 21:18  julio 30, 2022, 22:03   

                                         description                 due_at  \
0                                                  X  abril 20, 2022, 16:00   
1                                                  X                    

##### Checkear se hay filas duplicadas

In [4]:
print('Filas Duplicadas: ' + str(df.duplicated().any()) + '\n')

Filas Duplicadas: False



##### Info

In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16097 entries, 0 to 16096
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ticket_id                      16097 non-null  object 
 1   _fivetran_synced               16097 non-null  object 
 2   assignee_id                    10710 non-null  object 
 3   brand_id                       16097 non-null  object 
 4   created_at                     16097 non-null  object 
 5   updated_at                     16097 non-null  object 
 6   description                    16097 non-null  object 
 7   due_at                         518 non-null    object 
 8   group_id                       16097 non-null  object 
 9   external_id                    12035 non-null  object 
 10  is_public                      16097 non-null  object 
 11  organization_id                16071 non-null  object 
 12  priority                       15984 non-null 

##### Limpieza de tabla

In [6]:
# database clean
print('columns: ' + str(len(df.columns)))
print('rows: ' + str(len(df)))

#drop unnesessary columns
df.drop(['due_at', 'group_id', 'is_public', 'problem_id', 'type', 'source_from_id', 'source_from_title', 'source_rel', 'stg_zendesk__group → group_id', 'stg_zendesk__group → name'], axis=1, inplace=True)

print('limpieza de columnas exitosa')
print('columns: ' + str(len(df.columns)))
print('rows: ' + str(len(df)))

columns: 30
rows: 16097
limpieza de columnas exitosa
columns: 20
rows: 16097


In [7]:
print('columns: ' + str(len(df.columns)))
print('rows: ' + str(len(df)))

#Eliminar filas de migración
index_names = df[ df['description'] == 'Migracion Historica'].index
df.drop(index_names, inplace = True)

#eliminar filas con descripción = X
index_names = df[ df['description'] == 'X'].index
df.drop(index_names, inplace = True)

print('limpieza de filas exitosa')
print('columns: ' + str(len(df.columns)))
print('rows: ' + str(len(df)))


columns: 20
rows: 16097
limpieza de filas exitosa
columns: 20
rows: 4024


In [8]:
# Convertimos las fechas y horas importantes a datetimes para poder usarlas

months = [('enero', 'January'), ('febrero', 'February'), ('marzo', 'March'), ('abril', 'April'), ('mayo', 'May'), ('junio', 'June'), ('julio', 'July'), ('agosto', 'August'), ('septiembre', 'September'), ('octubre', 'October'), ('noviembre', 'November'), ('diciembre', 'December')]

for month in months:
    df['created_at'] = df['created_at'].str.replace(month[0], month[1])
    df['updated_at'] = df['updated_at'].str.replace(month[0], month[1])

df['created_at'] = pd.to_datetime(df['created_at'], format='%B %d, %Y, %H:%M')
df['updated_at'] = pd.to_datetime(df['updated_at'], format='%B %d, %Y, %H:%M')

print(df.head())

   ticket_id        _fivetran_synced        assignee_id           brand_id  \
2  1,301,191   abril 20, 2022, 15:19  4,623,907,810,715  4,705,210,809,883   
3  1,408,221    mayo 19, 2022, 14:12  4,623,907,810,715  4,705,210,809,883   
4  1,427,909   julio 31, 2022, 08:14  4,623,907,810,715  4,705,210,809,883   
5  1,420,812    julio 8, 2022, 02:11  4,623,907,813,019  4,705,210,809,883   
6  1,431,181  agosto 12, 2022, 04:13  4,623,907,810,715  4,705,210,809,883   

           created_at          updated_at  \
2 2022-04-12 15:42:00 2022-04-16 00:03:00   
3 2022-05-16 19:18:00 2022-05-19 14:04:00   
4 2022-07-27 21:18:00 2022-07-30 22:03:00   
5 2022-07-01 19:28:00 2022-07-07 23:03:00   
6 2022-08-09 16:05:00 2022-08-11 19:04:00   

                                         description external_id  \
2  Estimados\n\nJunto con saludar, favor solicito...         NaN   
3  Estimados\n\nJunto con saludar, favor solicito...         NaN   
4  Estimados\n\nJunto con saludar, favor solicito...    

In [14]:
def add_time(df):
    times = (df.updated_at - df.created_at) / pd.Timedelta('1 day')
    df['time'] = times

In [ ]:
#agregamos columna de tiempo
add_time(df)

##### Dividimos el dataframe en Varias secciones según tipo de documento

In [22]:
#Seleccionamos como subcontratos aquellos tickets que en su descripción contengan las siguientes palabras
subcontrato_general = ['sub', 'firma', 'firmas', 'logo', 'clausula', 'cláusula', 'anexo', 'avance', 'estado de pago']
orden_de_compra_general = ['orden de compra']
firmas = ['firma']
nuevos = ['nuevo contrato']

def get_df_by_words(df, words):
    df1 = df.loc[df['description'].str.contains('|'.join(words), case=False)]
    return df1

df_subcont = get_df_by_words(df, subcontrato_general)
df_firmas = get_df_by_words(df, firmas)
df_orden_de_compra = get_df_by_words(df, orden_de_compra_general)
df_nuevos = get_df_by_words(df, nuevos)

print(len(df_subcont))
print(len(df_firmas))
print(len(df_orden_de_compra))
print(len(df_nuevos))


1079
406
147
305


##### Análisis por fecha

In [23]:
# Sacamos el indicador timpo de respuesta/cantidad de tickets

def get_total_time(df):
    total_time = df.time.sum()
    return total_time

def get_total_tickets(df):
    total_tickets = len(df)
    return total_tickets

def get_average_time(df):
    average_time = get_total_time(df) / get_total_tickets(df)
    return average_time

print('General: ' + str(get_average_time(df)))
print('Subcontratos: ' + str(get_average_time(df_subcont)))
print('Firmas: ' + str(get_average_time(df_firmas)))
print('Orden de compra: ' + str(get_average_time(df_orden_de_compra)))
print('Nuevos: ' + str(get_average_time(df_nuevos)))


General: 5.633846400762094
Subcontratos: 5.0823376840696115
Firmas: 4.8499555281882865
Orden de compra: 4.473110355253213
Nuevos: 14.353290072859746


In [24]:
# Sacmaos el top 10 de los tickets con más tiempo de respuesta

def get_top_10(df):
    df = df.sort_values(by='time', ascending=False)[['ticket_id', 'description', 'time']]
    df = df.head(10)
    return df

print(get_top_10(df))

      ticket_id                                        description       time
3810  1,302,522  Configuración de nuevo contrato | MENTE CREATI...  94.030556
3825  1,300,986  Configuración de nuevo contrato | BODENOR FLEX...  89.681944
2940  1,303,046  Estimados, \n\nFavor configurar formatos PDf y...  88.988194
3581  1,303,929  Configuración de nuevo contrato | Constructora...  86.954861
3367  1,405,394  Baja Cliente EMPRESA CONSTRUCTORA TRESCO SPA |...  84.952083
3280  1,305,772  Configuración de nuevo contrato | SOCIEDAD CON...  83.003472
3519  1,300,838  Baja Cliente REPRESENTACIONES CANONTEX LIMITAD...  73.825694
3701  1,409,796  Configuración de nuevo contrato | SIGMA CONSTR...  67.974306
1587  1,305,520  Estimados:\n\n \n\nTenemos un cambio en Socove...  66.971528
3902  1,404,340  Configuración de nuevo contrato | SOCIEDAD ENE...  66.852778


##### Exportación a Excel

In [25]:
# Exportamos a csv

df.sort_values(by='time', ascending=False)[['ticket_id', 'description', 'time']].to_csv('tickets_time.csv', index=False)